<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir=rtl>
<h3><center>تمرین سوم درس پردازش زبان‌های طبیعی</center></h3>
<h4><center>چالش جست‌وجوی دارو</center></h4>
<table width='100%' style="border: none;">
    <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>علیرضا بلال</h5></td>
        <td style="border: none;"><h5>زهرا رجالی</h5></td>
        <td style="border: none;"><h5>جواد راضی</h5></td>
    </tr>
        <tr style="border: none; text-align: center;">
        <td style="border: none;"><h5>4002XXYYY</h5></td>
        <td style="border: none;"><h5>401204716</h5></td>
        <td style="border: none;"><h5>401204354</h5></td>
    </tr>
</table>
<h5 style="font-size: 16px;"><center> بهار ۱۴۰۲ </center></h5>
<br/>
<hr/>
<br/>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar', 'B Lotus', 'Calibri'" size=3><div dir='rtl' align='justify'>
<b>
    فایل ژوپیتر این تمرین در کولب توسعه داده و تست شده‌است. این فایل هم در محیط کولب، هم با ایمیج داکر jupyter/datascience-notebook تست شده‌است و همه قطعه‌کدها خروجی مورد انتظار را می‌دهند. اگه در بازتولید خروجی بعضی سل‌ها، یا کدهای تمرین مشکلی وجود داشت، ممنون می‌شویم در صورت امکان به ما اطلاع دهید تا فایل را در محیطی که قابل اجرا است، اجرا نموده و خروجی را نمایش دهیم. 
</b>

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **خلاصه‌ای از صورت مسئله و راه‌حل**
اینجا یه خلاصه می‌دیم که مسئله چیه. بعد یه outline از کارایی که کردیم می‌دیم. 

کارایی که کردیم (حالا به شکل بهتری بیان میشن):‌
۱- یافتن دیتابیسی از داروهای انگلیسی
۲- نرمالیزه کردن دیتابیس به شکلی که می‌خوایم 
۳- یافتن معادل فارسی داروهای دیتابیس 


<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#    **مراحل طراحی سامانه جستجوی دارو**
در این بخش، مراحلی که برای طراحی سامانه جستجوی دارو طی شده‌اند و در قسمت پیشین خلاصه‌ای از آن بیان شد، به تفصیل شرح داده خواهد شد. 



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
### ۱. یافتن دیتابیس داروها به زبان انگلیسی

در وب‌سایت سازمان غذا و داروی ایران، دیتاستی با حدود ۵۰ هزار ورودی که برای هر دارو شامل نام دارو به فارسی و لاتین، نام جنریک آن، نام برند و موارد دیگر می‌باشد، برای دانلود در دسترس عام است. دیتابیس اصلی استفاده شده در این تمرین، این دیتابیس می‌باشد. البته بر روی این دیتابیس، تغییرات و نرمال‌سازی‌هایی انجام شده‌است که در ادامه ذکر خواهند شد. 
دیتابیس دیگر، از این مخزن گیت‌هاب دانلود شد. این دیتابیس، شامل نام هر دارو به ازای کد ATC آن می‌باشد. کد ATC دارو، که در دیتاست سازمان غذا و داروی ایران نیز موجود است، طبقه‌بندی داروها رو توصیف می‌کند. برای مثال، برای داروی ملاتونین که یک قرص تنظیم ساعت خواب است، کد ATC آن N05CH01 می‌باشد. در این کد، کاراکترها به ترتیب، سلسله‌مراتب دسته‌ای که دارو به آن متعلق است را مشخص می‌کنند. برای داروی ذکرشده، این سلسله مراتب به شرح زیر است:
- N: سیستم عصبی
- N05: داروهای روان‌پزشکی
- N05C: خواب‌آور و آرام‌بخش
- N05CH: آگونیست گیرنده‌های ملاتونین
- N05CH01: قرص ملاتونین ۲ میلی‌گرم

همان‌طور که از این مثال پیداست، کد ATC دارو اطلاعات ارزشمندی در خصوص دسته‌بندی، و کارکرد آن به ما می‌دهد. با توجه به اینکه دیتاست اصلی سازمان غذا و دارو، برای هر دارو کد ATC آن را نیز دارد، با Join نمودن این دو دیتاست، توانستیم برای هر دارو، دسته‌بندی و کاربردهایی از آن را در قالب چند ستون اضافه، به دیتاست اضافه نماییم. 

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۲. نرمال‌سازی دیتابیس
اینجا توضیح می‌دیم که دیتابیس چرا و چطوری نرمال شده. یه سری ستون که نیاز نداشتیم حذف شدن. ستون اسم برند شامل چند تا برند می‌تونه باشه که سطرایی که این ستون رو دارن، به چند سطر تقسیم شدن. کد و فلان هم اضافه میشه احتمالا. 

In [1]:
## TODO 
pass 

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۳. ترکیب دیتاست داروها با دیتاست طبقه‌بندی کد ATC داروها
اینجا توضیح میدیم دو تا دیتاست چجوری جوین شدن

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## ۴. جداسازی دیتابیس‌های فارسی و انگلیسی
توضیحات در خصوص تقسیم دیتابیس به دو تا دیتابیس فارسی و انگلیسی

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
# **یافتن و تصحیح ایرادات هه کسره**
<br>

در نوشتار فارسی، خطای ه‌کسره با الگوهای خاصی رخ می‌دهد. به طور کلی،‌این الگوها تا حد زیادی قاعده‌مند می‌باشند و با وجود استثنائات، می‌توان خطاهای «ه‌کسره» را به چند دسته خاص تقسیم کرد.  
در این [بلاگ‌پست](https://blog.irandargah.com/%D8%BA%D9%84%D8%B7%E2%80%8C%D9%87%D8%A7%DB%8C-%D9%86%DA%AF%D8%A7%D8%B1%D8%B4%DB%8C-%D9%88-%D8%A7%D9%85%D9%84%D8%A7%DB%8C%DB%8C%D8%8C-%D9%82%D8%A7%D8%AA%D9%84-%D8%A7%D8%B9%D8%AA%D8%A8%D8%A7%D8%B1/) 
به دسته‌بندی‌های مختلف خطای هکسره در زبان فارسی اشاره مختصر و مفیدی شده‌است. در این تمرین نیز پیاده‌سازی تشخیص هکسره را بر مبنای منابع از این دست انجام داده‌ایم. در ادامه به طور مختصر الگوهای رایج این خطای املایی شرح داده می‌شوند. 

#### الگوهای رایج غلط ه‌کسره
1. **صفت و موصوف یا مضاف و مضافه‌الیه**  
در ترکیبات وصفی و اضافی، باید بعد از موصوف یا مضاف الیه، از کسرهٔ اضافهٔ (ــِ) بین دو کلمهٔ مورد نظر استفاده کرد. بهتر است از "ه" یا "ـه" در این نوع ترکیبات استفاده نشود. 
    * **استثنا:‌ زمانی که تکواژ ه بخشی از واژه است**  
    در موارد دیگری که ممکن است با خطاهای هکسره در فارسی مواجه شویم، حالتی وجود دارد که "ه/ ـه" به عنوان حرف اصلی در انتهای واژه ظاهر شده و جزئی از واژه است، و به این نوع از "ه" هم "ه غیر ملفوظ" گفته می‌شود. در این حالت‌ها، "ه/ ـه" نباید حذف شود و استفاده از کسره بجای آن مناسب نیست. 
<br><br>

2. **تکواژ ه به عنوان معرفه‌ساز**  
معرفه استفاده می‌شود. اسم‌های معرفه، اسامی هستند که برای شناسایی کسی یا چیزی به کار می‌روند. در صورتی که نویسنده در متن خود می‌خواهد به کسی یا چیزی اشاره کند که او را می‌شناسد، تکواژ "ه" به آخر کلمات اضافه می‌شود. استفاده از هکسره (-ِ) در این موارد کاملاً نادرست است. 

3. **تکواژ ه به جای فعل**  
در زبان محاوره‌ای ما در برخی موارد به جای فعل "است" یا "هست"، از صدای "e" استفاده می‌کنیم. در این حالت‌ها باید از حرف "ه" به جای کسره استفاده کرد.
همچنین در زبان محاوره، برای سوم شخص فعل‌ها، گاهی به جای پایان با "ـَد" از صدای "e" استفاده می‌شود. در چنین مواقعی نیز باید از حرف "ه" به جای کسره استفاده کرد.



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
## **پیاده‌سازی سامانه تشخیص خطای هکسره**
<br>

#### کلاس `HeKasraCorrection`
یک آبجکت از این کلاس، متنی که پیش‌پردازش‌های قبل روز آن انجام گرفته شده (به همراه متن خالص اولیه) را در خود دارد. متدهای آبجکت این کلاس، توسط متدهای پایپ‌لاین تشخیص هکسره صدا زده می‌شوند. اگر در پایپ‌لاین، متدی تشخیص داد که خطای هکسره وجود دارد، با تابع `vote_for_correction` این خطا به خطاهای بالقوه ارسال می‌شود.آرگومان ‍‍`order` در این تابع اولویت این تصحیح خطا را بیان می‌کند.  
<br> 
تابع ‍‍‍`veto_correction` در این کلاس، اگر توسط کامپوننتی در پایپ‌لاین زده‌شد، خطاهای تشخیص داده‌شده توسط کامپوننت‌های قبلی پایپ‌لاین را وتو می‌کند. به عنوان مثال، در ترکیب «خانه زیبا» تابعی ابتدایی در پایپ‌لاین تشخیص خطای هکسره در این ترکیب وصفی می‌دهد، اما در ادامه پایپ‌لاین، تابعی تشخیص می‌دهد که ه، بخشی از خود کلمه «خانه» است و خطای هکسره محسوب نمی‌شود. بنابراین تشخیص خطای قبلی در اینجا وتو می‌گردد. 
<br>  

تابع `finalize` نیز در انتها، پس از آنکه تمام ماژول‌های پایپ‌لاین رای خود را در خصوص خطاهای هکسره متن دادند، این خطاها را جمع‌آوری کرده، و به ترتیب اولویت آن‌ها را اعمال می‌کند. در نهایت نیز متن تصحیح شده ایجاد گردیده و خطاها و بازه مربوط به هر خط نیز مشخص می‌شود. 

In [228]:
from collections import defaultdict


class HeKasraCorrection: 
    def __init__(self, processed_text):
        self.processed_text = processed_text
        self.corrections = {
            'correct': processed_text['raw_text'],
        }
        self.correction_judgements = defaultdict(list)
    
    def vote_for_correction(self, invalid_token, corrected_token, str_index, order=10):
        self.correction_judgements[str_index].append({
            'invalid_token': invalid_token,
            'corrected_token': corrected_token,
            'str_index': str_index,
            'order': order,
        })
        return self.correction_judgements[str_index]
    
    def veto_correction(self, already_correct_token, str_index):
        self.correction_judgements[str_index].append({
            'invalid_token': already_correct_token,
            'corrected_token': already_correct_token,
            'str_index': str_index,
            'order': 0,
        })
        return self.correction_judgements[str_index]
 
    def apply_correction_judgements(self, token, str_index):
        judgements = self.correction_judgements[str_index]
        if len(judgements) == 0:
            return
        
        sorted_judgements = sorted(judgements, key=lambda x: x['order'])
        prioritized_correction = sorted_judgements[0]
        corrected_form = self.corrections['correct'][:str_index] + prioritized_correction['corrected_token'] + self.corrections['correct'][str_index+len(prioritized_correction['invalid_token']):]
        self.corrections['correct'] = corrected_form
        if token != prioritized_correction['corrected_token']:
            self.corrections[prioritized_correction['invalid_token']] = [int(str_index), int(str_index)+len(prioritized_correction['invalid_token'])] 
              
    def finalize(self):
        for str_index in self.correction_judgements.copy().keys():
            self.apply_correction_judgements(self.correction_judgements['invalid_token'], str_index)
        if self.corrections['correct'] == self.processed_text['raw_text']:
          self.corrections = {}
        return {
            **self.processed_text,
            'correction': self.corrections,
        }       

In [229]:
from dadmatools.embeddings import get_embedding
# Some downloading, so separate the cell
embeddings = get_embedding('word2vec-conll')

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>
#### کلاس `HeKasraExtractor`
این کلاس، در واقع کلاس اصلی سرویس است که با دریافت یک متن فارسی، خطاهای ه‌کسره آن را دریافت کرده و متن تصحیح‌شده را، به همراه بازه دقیق خطاها برمی‌گرداند. برای تشخیص ه‌کسره، تابع `run` این کلاس پایپ‌لاینی را اجرا می‌کند که شامل پیش‌پردازش‌هایی روی متن، annotate کردن آن و یافتن انواغ مختلف غلط ه‌کسره می‌باشد.  
<br> 


In [230]:
from hazm import WordTokenizer, POSTagger, Normalizer, InformalNormalizer, SentenceTokenizer, Lemmatizer
import re

normalizer = Normalizer()
inf_normalizer = InformalNormalizer(seperation_flag=True)
sent_tokenizer = SentenceTokenizer
tokenizer = WordTokenizer(join_verb_parts=True)
tagger = POSTagger(model="./model/postagger.model")


In [231]:

import math 

class HeKasraExtractor:
    def __init__(self):
        pass
    
    def preprocess(self, text):
        normalized_text = normalizer.normalize(text)
        tokens = tokenizer.tokenize(normalized_text)
        tagged_tokens = tagger.tag(tokens)

        return {
            'raw_text': text,
            'pipe_text': text,
            'normalized_text': normalized_text,
            'tokens': tokens,
            'pos_tags': tagged_tokens
        }


    # Check he_kasra error in adjective/adverb
    def vote_n_adj_he_kasra(self, he_kasra_correction, processed_text):
        pos_pairs = zip(processed_text['pos_tags'][:-1], processed_text['pos_tags'][1:])


        for ppair in pos_pairs:
          p1, p2 = ppair
          token_1, tag_1 = p1
          token_2, tag_2 = p2        

          first_token_roles = ('N', 'Ne', 'PRO', 'AJ', 'AJe')
          second_token_roles = ('N', 'Ne', 'AJ', 'PRO', 'AJe')

          if tag_1 in first_token_roles and tag_2 in second_token_roles:
              if token_1.endswith('ه'):
                he_kasra_correction.vote_for_correction(token_1, token_1[:-1], processed_text['raw_text'].index(token_1))


    def check_word_contains_he(self, word, next_word, text):
        normalized_word = normalizer.normalize(word)

        if not normalized_word.endswith('ه'):
            return False
        
        vocab = embeddings.get_vocab()
        plural_form = word + 'ها'
        if plural_form not in vocab:
          return False

        return True
    
    def check_sent_has_verb(self, processed_text):
      return 'V' in [t[1] for t in processed_text['pos_tags']]

    def check_sent_has_too_many_res(self, processed_text):
      is_res = [r[1] == 'RES' for r in processed_text['pos_tags']]
      ratio = is_res.count(True)/len(is_res)
      return ratio > 0.75 

    # Check if sentence should have had 'he' at the end of the last word, to act as a verb
    def check_sent_should_have_he_as_verb(self, he_kasra_correction, processed_text):
      if self.check_sent_has_verb(processed_text):
        return False
      if not self.check_sent_has_too_many_res(processed_text):
        return False

      new_sent = processed_text['raw_text'] + 'ه'
      new_pr = self.preprocess(new_sent)
      if self.check_sent_has_verb(new_pr) and not self.check_sent_has_too_many_res(new_pr):
        latest_token = processed_text['pos_tags'][-1][0]
        he_kasra_correction.vote_for_correction(latest_token, latest_token + 'ه', processed_text['raw_text'].index(latest_token))
        processed_text['pos_tags'] = new_pr['pos_tags']
        processed_text['tokens'] = new_pr['tokens']


    # If 'he' character is part of the word, then it's not a he_kasra error and should not be corrected
    def veto_if_word_he_part_of_word(self, he_kasra_correction, processed_text):
        pos_pairs = zip(processed_text['pos_tags'][:-1], processed_text['pos_tags'][1:])


        for ppair in pos_pairs:
          p1, p2 = ppair
          token_1, tag_1 = p1
          token_2, tag_2 = p2        

          contains_he = self.check_word_contains_he(token_1, token_2, processed_text)
          if contains_he:
              he_kasra_correction.veto_correction(token_1, processed_text['raw_text'].index(token_1))                                
                                
    def run(self, input_sentence):
        prep_text = self.preprocess(input_sentence)
        he_kasra_correction = HeKasraCorrection(prep_text)
        pipe = [
            self.check_sent_should_have_he_as_verb,
            self.vote_n_adj_he_kasra, 
            self.veto_if_word_he_part_of_word,
        ]

        for func in pipe:
            func(he_kasra_correction, prep_text)
            
        result = he_kasra_correction.finalize()
        return result['correction']
    



<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## ارزیابی عملکرد سامانه 
قطعه کد زیر، با ایجاد یک اینستنس از کلاس ‍`HeKasraExtractor` برای تعدادی ورودی نمونه عملکرد آن را می‌سنجد. آبجکت خروجی تابع ران برای هر یک از ورودی‌ها در خروجی این سلول چاپ‌شده است. همان‌طور که مشاهده می‌شود، برای انوع خطاهای هکسره در ترکیبات وصفی و اضافی، ه معرفه‌ساز، ه به عنوان فعل و ...، کلاس پیاده‌سازی شده عملکرد صحیحی را دارد. 

In [232]:
import json
import numpy as np
hkasra_extractor = HeKasraExtractor()
input_samples = [
    {
      'text_input': 'کوروشه کبیر',
      'expected_corrected_text': 'کوروش کبیر' ,
      'correct_input': False
    },
        {
      'text_input': 'حال من خوب است.',
      'expected_corrected_text': 'حال من خوب است.' ,
      'correct_input': True
    },
        {
      'text_input': 'حاله من خوبه',
      'expected_corrected_text': 'حال من خوبه' ,
      'correct_input': False
    },  
    {
      'text_input': 'حاله من خوب',
      'expected_corrected_text': 'حال من خوبه' ,
      'correct_input': False
    },
    {
      'text_input': 'من اگه کتابه تو رو داشتم',
      'expected_corrected_text': 'من اگه کتاب تو رو داشتم' ,
      'correct_input': False
    },  {
      'text_input': 'پسره داشت میرفت مدرسه',
      'expected_corrected_text': 'پسره داشت میرفت مدرسه' ,
      'correct_input': True
    },  {
      'text_input': 'این دختره دیوانه کار دستمون داد',
      'expected_corrected_text': 'این دختر دیوانه کار دستمون داد' ,
      'correct_input': False
    },
        {
      'text_input': 'گل زیبا',
      'expected_corrected_text': 'گل زیبا' ,
      'correct_input': True
    },
        {
      'text_input': 'گله زیبایی را تقدیم کردم',
      'expected_corrected_text': 'گل زیبایی را تقدیم کردم' ,
      'correct_input': False
    },
        {
      'text_input': 'درختِ بزرگ',
      'expected_corrected_text': 'درختِ بزرگ' ,
      'correct_input': True
    },
        {
      'text_input': 'این کتاب خوبه',
      'expected_corrected_text': 'این کتاب خوبه' ,
      'correct_input': True
    },    {
      'text_input': 'دستش خیلی تنده',
      'expected_corrected_text': 'دستش خیلی تنده' ,
      'correct_input': True
    },
        {
      'text_input': 'فرشه خیلی قشنگ بود.',
      'expected_corrected_text': 'فرشه خیلی قشنگ بود.' ,
      'correct_input': True
    },    {
      'text_input': 'بسه دیگه خسته شدم.',
      'expected_corrected_text': 'بسه دیگه خسته شدم.' ,
      'correct_input': True
    },    {
      'text_input': 'خورشیده طلایی رنگ طلوع کرد.',
      'expected_corrected_text': 'خورشید طلایی رنگ طلوع کرد.' ,
      'correct_input': False
    },
        {
      'text_input': 'یه سر به پیجِ ما بزنید',
      'expected_corrected_text': 'یه سر به پیجِ ما بزنید' ,
      'correct_input': True
    },
        {
      'text_input': 'اون خیابونه رو بستن جدیداً.',
      'expected_corrected_text': 'اون خیابونه رو بستن جدیداً.' ,
      'correct_input': True
    },    {
      'text_input': 'علی پرروئه',
      'expected_corrected_text': 'علی پرروئه' ,
      'correct_input': True
    },
        {
      'text_input': 'اصلاً نمی‌فهمم از چیه من خوشش اومد!',
      'expected_corrected_text': 'اصلاً نمی‌فهمم از چی من خوشش اومد!' ,
      'correct_input': False
    },
    {
      'text_input': 'اون اصلا غذا نمی­خورِ',
      'expected_corrected_text': 'اون اصلا غذا نمیخوره' ,
      'correct_input': False
    },
    {
      'text_input': 'برکه‌ی مشهور',
      'expected_corrected_text': 'برکه‌ی مشهور' ,
      'correct_input': True
    },
    {
      'text_input': 'فرشته مرگ',
      'expected_corrected_text': 'فرشته مرگ' ,
      'correct_input': True
    }
]

evaluation = np.zeros((len(input_samples), 5), dtype=object)
for index, sample in enumerate(input_samples):
  response = hkasra_extractor.run(sample['text_input'])
  corrected_text = response['correct'] if 'correct' in response else sample['text_input']
  print('Text Input: %s' % sample['text_input'])
  print('Service Response', response)
  print('********')
  evaluation[index] = [sample['text_input'], sample['expected_corrected_text'], corrected_text, sample['expected_corrected_text'] == corrected_text, sample['text_input'] == sample['expected_corrected_text']]

Text Input: کوروشه کبیر
Service Response {'correct': 'کوروش کبیر', 'کوروشه': [0, 6]}
********
Text Input: حال من خوب است.
Service Response {}
********
Text Input: حاله من خوبه
Service Response {'correct': 'حال من خوبه', 'حاله': [0, 4]}
********
Text Input: حاله من خوب
Service Response {'correct': 'حال من خوبه', 'خوب': [8, 11], 'حاله': [0, 4]}
********
Text Input: من اگه کتابه تو رو داشتم
Service Response {}
********
Text Input: پسره داشت میرفت مدرسه
Service Response {}
********
Text Input: این دختره دیوانه کار دستمون داد
Service Response {'correct': 'این دختر ددیوانهکار دستمون داد', 'دختره': [4, 9], 'دیوانه': [10, 16]}
********
Text Input: گل زیبا
Service Response {}
********
Text Input: گله زیبایی را تقدیم کردم
Service Response {'correct': 'گل زیبایی را تقدیم کردم', 'گله': [0, 3]}
********
Text Input: درختِ بزرگ
Service Response {}
********
Text Input: این کتاب خوبه
Service Response {}
********
Text Input: دستش خیلی تنده
Service Response {}
********
Text Input: فرشه خیلی قشنگ بود.
Ser

<font face="'vazirmatn', 'Vazir', 'B Nazanin', 'XB Zar'" size=4><div dir='rtl' align='justify'>

## نتایج روی داده‌های تست 
قطعه کد زیر،‌ جدولی را چاپ می‌کند که به ازای هر ورودی، خروجی مورد انتظار (صحیح) و خروجی سرویس را نمایش می‌دهد. در سلول بعدی نیز دقت مدل در تشخیص وجود یا عدم وجود خطای هکسره سنجیده می‌گردد. 

In [233]:
import pandas as pd
evaluation_df = pd.DataFrame(evaluation, columns=['Raw_Input', 'Expected_Output', 'Model_Output', 'Correct_Prediction', 'No_HeKasra_Error'])
evaluation_df

,Raw_Input,Expected_Output,Model_Output,Correct_Prediction,No_HeKasra_Error
0,کوروشه کبیر,کوروش کبیر,کوروش کبیر,True,False
1,حال من خوب است.,حال من خوب است.,حال من خوب است.,True,True
2,حاله من خوبه,حال من خوبه,حال من خوبه,True,False
3,حاله من خوب,حال من خوبه,حال من خوبه,True,False
4,من اگه کتابه تو رو داشتم,من اگه کتاب تو رو داشتم,من اگه کتابه تو رو داشتم,False,False
5,پسره داشت میرفت مدرسه,پسره داشت میرفت مدرسه,پسره داشت میرفت مدرسه,True,True
6,این دختره دیوانه کار دستمون داد,این دختر دیوانه کار دستمون داد,این دختر ددیوانهکار دستمون داد,False,False
7,گل زیبا,گل زیبا,گل زیبا,True,True
8,گله زیبایی را تقدیم کردم,گل زیبایی را تقدیم کردم,گل زیبایی را تقدیم کردم,True,False
9,درختِ بزرگ,درختِ بزرگ,درختِ بزرگ,True,True


In [234]:
accuracy = evaluation_df['Correct_Prediction'].mean()
he_kasra_acc = evaluation_df.query('No_HeKasra_Error == False')['Correct_Prediction'].mean()
he_kasra_free_acc = evaluation_df.query('No_HeKasra_Error == True')['Correct_Prediction'].mean()

print("Model Accuracy: %1f" % accuracy)
print("Model Accuracy When HeKasra Error Occured: %1f" % he_kasra_acc)
print("Model Accuracy When Input Was HeKasra Error Free: %1f" % he_kasra_free_acc)

Model Accuracy: 0.863636
Model Accuracy When HeKasra Error Occured: 0.666667
Model Accuracy When Input Was HeKasra Error Free: 1.000000
